In [5]:
import argilla as rg
from datasets import load_dataset
import os
import sys
from pathlib import Path
module_path = Path(os.path.abspath("")).parent
print(module_path)
sys.path.append(str(module_path))
import utils as utils
from dotenv import load_dotenv
load_dotenv()

argilla_api_url = os.getenv("argilla_api_url")
argilla_api_key = os.getenv("argilla_api_key")

client = rg.Argilla(
  api_url=argilla_api_url,
  api_key=argilla_api_key
)



/home/abdelrahman.sadallah/mbzuai/review_rewrite


In [6]:
client.me.username

'boda'

In [7]:
##### Create Admin user
owner_user = rg.User(
    username="tim",
    first_name="Tim",
    last_name="Baumgärtner",
    role="owner",
    password="123456789",
    client=client
)

if not client.users(owner_user.username):
    owner_user.create()

In [8]:
BATCH = 10

NUM_ANNOTATORS = 5
credentials_file = f'../data/human_annotation_{BATCH}/annotators_credentials.txt'

## Create a list of uesrnames and passwords with 
user_data = utils.generate_username_password_list(NUM_ANNOTATORS)
utils.save_to_file( user_data, credentials_file)


In [9]:
### Create annotator Users

annotators = []

annotators_credentials = utils.read_from_file(credentials_file)

assert len(annotators_credentials) == NUM_ANNOTATORS


# delete all users and workspaces
for user in client.users:
    if user.role == 'annotator':
        user.delete()
        
# Delete all datasets
for ds in client.datasets:
    ds.delete()
for workspace in client.workspaces:
    workspace.delete()

for i in range(NUM_ANNOTATORS):
    
    ## Create user

    annotator_user = rg.User(
        username=annotators_credentials[i]['username'],
        password=annotators_credentials[i]['password'],
        role="annotator",
        client=client
    )

    annotator_user.create()

    ## create workspace per uesr
    if not client.workspaces(annotator_user.username):
        rg.Workspace(
            name=annotator_user.username,
            client=client
        ).create()
        print(f"Created workspace for {annotator_user.username}")

    ## add user to workspace
    client.workspaces(annotator_user.username).add_user(annotator_user.username)

    ##
    annotators.append({'user': annotator_user, 'workspace': client.workspaces(annotator_user.username)})

## Create a workspace for the owner
if not client.workspaces(owner_user.username):
    rg.Workspace(
        name=owner_user.username,
        client=client
    ).create()
    print(f"Created workspace for {owner_user.username}")

## Create a workspace for the Boda
if not client.workspaces('boda'):
    rg.Workspace(
        name='boda',
        client=client
    ).create()
    print(f"Created workspace for boda")
    

/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/argilla/client.py:253: UserWarning: Workspace with name 'qhBkNx6n' not found.
  warnings.warn(f"Workspace with name {name!r} not found.")


Created workspace for qhBkNx6n


/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/argilla/client.py:253: UserWarning: Workspace with name 'cYVX6CPX' not found.
  warnings.warn(f"Workspace with name {name!r} not found.")


Created workspace for cYVX6CPX


/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/argilla/client.py:253: UserWarning: Workspace with name 'whnmAGJS' not found.
  warnings.warn(f"Workspace with name {name!r} not found.")


...

Created workspace for RRgk02HS


/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/argilla/client.py:253: UserWarning: Workspace with name 'tim' not found.
  warnings.warn(f"Workspace with name {name!r} not found.")


Created workspace for tim


/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/argilla/client.py:253: UserWarning: Workspace with name 'boda' not found.
  warnings.warn(f"Workspace with name {name!r} not found.")


Created workspace for boda


In [10]:
### Define Fields 
fields = [
rg.TextField(
    name="review_point",
    title="Review Point",
    use_markdown=False,
    required=True,
    description="Review point extracted from the full review. You are only required to evalutae this specific review poin, and not the full review.",
),
# rg.TextField(
#     name="review",
#     title="Review",
#     use_markdown=False,
#     required=True,
#     description="Full review text. Only use this if you needed context to evaluate the review point.",
# ),
]

In [11]:
questions = [
rg.LabelQuestion(
    name="actionability",
    labels={
    "1": "Unactionable", 
    "2": "Borderline Actionable", 
    "3": "Somewhat Actionable", 
    "4": "Mostly Actionable", 
    "5": "Highly Actionable"
    },
    title="What is the Actiobability of the Review Point?",
    description='''
Actionability refers to the clarity and specificity of guidance provided in review comments, ensuring authors know what actions to take to improve their work. Highly actionable feedback is both explicit and concrete, offering direct suggestions or clearly defined steps that authors can easily implement, whereas less actionable feedback may be vague, implicit, or leave authors uncertain about how to proceed. 
    ''',
    required=True,
    visible_labels=None
),


rg.LabelQuestion(
    name="grounding_specificity",
    labels={
        "1": "Not Grounded",
        "2": "Weakly grounded and not specific",
        "3": "Weakly grounded and specific",
        "4": "Fully grounded and not specific",
        "5": "Fully grounded and specific"},
    title="How Grounded and Specific is the Review Point?",
    description='''Grounding and specificity refer to how clearly a review comment identifies the specific part of a paper being addressed (grounding) and how detailed the comment is in explaining the issue or suggestion (specificity). These aspects are crucial for ensuring authors can accurately pinpoint and address the feedback, improving the quality and relevance of revisions.''',
    required=True,
    visible_labels=5),

rg.LabelQuestion(
    name="verifiability",
    labels={
        "1": "Unverifiable",
        "2": "Borderline verifiable",
        "3": "Somewhat verifiable",
        "4": "Mostly verifiable",
        "5": "Fully verifiable",
        "X": "No Claim"},
    title="How Verifiable is the Review Point?",
    description='''Verifiability refers to assessing whether a comment contains a claim or subjective opinion and, if so, how well that claim is supported with justification. Justification can include logical reasoning, common-sense arguments, or references, ensuring the claim is clear, substantiated, and helpful for authors to act upon.''',
    required=True,
    visible_labels=6),

rg.LabelQuestion(
    name="helpfulness",
    labels={
        "1": "The comment is not helpful at all",
        "2": "The comment is barely helpful",
        "3": "The comment is somewhat helpful",
        "4": "The comment is mostly helpful",
        "5": "The comment is highly helpful"},
    title="How Helpful is the Review Point?",
    description='''
Helpfulness measures the value of a review comment to authors in improving their work, rated on a scale from 1 (not helpful at all) to 5 (highly helpful). Higher scores correspond to comments that identify weaknesses with clear, actionable suggestions, while lower scores reflect vague or unconstructive feedback that offers little guidance.''',
    required=True,
    visible_labels=5),

rg.LabelQuestion(
    name="professional_tone",
    labels={
        "0": "No",
        "1": "Yes",},
    title="Is the review conducted in a Professional Tone?",
    description='''
Professional Tone assesses the formality, respect, and clarity of language in peer reviews, ensuring constructive, respectful, and impartial feedback that fosters collaboration and focuses on the work rather than the individual.''',
    required=False,
    # visible_labels=2
    ),

rg.LabelQuestion(
    name="valid_point",
    labels={
        "1": "Yes",
        "0": "No",},
    title="Is this review point not complete (it can not be evaluated)?",
    description="If the review point is not complete, please select 'Yes'. Otherwise, select 'No'.",
    required=False,
),
rg.LabelQuestion(
    name="addressed_to_author",
    labels={
        "authors": "Authors",
        "meta_reviewer": "Meta Reviewer",},
    title="Is this comment addressed to the authors or the meta-reviewer?",
    description="Who do you think the comment is addressed to?",
    required=False,
)
]

In [12]:
## read a markdown file and store it into string
with open('/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/guidelines_short.md', 'r') as file:
    guidelines =  file.read()

In [13]:

settings = rg.Settings(
    fields=fields,
    questions=questions,
    guidelines=guidelines,
    allow_extra_metadata=True,
    distribution=rg.TaskDistribution(min_submitted=1),
)



In [14]:
hf_dataset = load_dataset("boda/review_evaluation_main_8", split="train")

## take only first 25 entries 
# hf_dataset = hf_dataset.select(range(50))

## save the chosen ids to a file


README.md:   0%|          | 0.00/385 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/254k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [15]:

# hf_dataset = hf_dataset.remove_columns(column_names=['paper_id', 'venue','actionability',
#  'grounding_specificity',
#  'verifiability',
#  'helpfulness',
#  'appropriateness',
#  'Do you need full review?'])

In [16]:
DATA = client.datasets(name="review_evaluation")


## Delete all datasets
# for ds in client.datasets:
#     ds.delete()

### Create dataset for each user
for i in range(NUM_ANNOTATORS):
    dataset = rg.Dataset(
        name=f"review_evaluation_{annotators[i]['user'].username}",
        workspace=annotators[i]['workspace'],
        settings=settings,
    )
    dataset.create()
    dataset.records.log(records=hf_dataset, mapping={"point": "review_point"})
    # dataset.records.log(records=hf_dataset, mapping={"focused_review": "review", "point": "review_point"})

## Create dataset for Boda, and Tim
dataset = rg.Dataset(
    name=f"review_evaluation_tim",
    workspace=client.workspaces('tim'),
    settings=settings,
)
dataset.create()
dataset.records.log(records=hf_dataset, mapping={"point": "review_point"})
dataset = rg.Dataset(
    name=f"review_evaluation_boda",
    workspace=client.workspaces('boda'),
    settings=settings,
)
dataset.create()
dataset.records.log(records=hf_dataset, mapping={"point": "review_point"})





/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/argilla/client.py:354: UserWarning: Dataset with name 'review_evaluation' not found in workspace 'qhBkNx6n'
  warnings.warn(f"Dataset with name {name!r} not found in workspace {workspace.name!r}")
/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/argilla/records/_io/_datasets.py:265: UserWarning: Record id column not found in Hugging Face dataset. Using row index and split for record ids.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/argilla/records/_mapping/_mapper.py:89: UserWarning: Keys ['paper_id', 'venue', 'focused_review', 'point'] in data are not present in the mapping and will be ignored.
  warnings.warn(f"Keys {unknown_keys} in data are not present in the mapping and will be ignored.")


DatasetRecords: The provided batch size 256 was normalized. Using value 200.

Sending records...: 100%|██████████| 1/1 [00:13<00:00, 13.31s/batch]


...

DatasetRecords: The provided batch size 256 was normalized. Using value 200.

Sending records...: 100%|██████████| 1/1 [00:11<00:00, 11.27s/batch]


DatasetRecords: The provided batch size 256 was normalized. Using value 200.

Sending records...: 100%|██████████| 1/1 [00:11<00:00, 11.87s/batch]


DatasetRecords(Dataset(id=UUID('ab76ff0d-cc81-45a7-8034-426fe507e68a') inserted_at=datetime.datetime(2025, 3, 19, 11, 21, 37, 939735) updated_at=datetime.datetime(2025, 3, 19, 11, 21, 41, 337130) name='review_evaluation_boda' status='ready' guidelines="### General instructions:  \n- For each aspect, classify the review point to the most suitable category it belongs to.  \n- The review point you are evaluating is addressed to the draft’s authors, who have the best knowledge of the draft's content.  \n- The primary purpose of the review is to help/guide authors in improving their drafts. Keep this in mind while evaluating the review point. Whenever you encounter a borderline case, think: *“Will this review point help authors improve their draft?”*  \n- If you don’t have enough context to judge the review point or the review point is not complete, then choose the most suitable labels and mark the review as not complete.  \n- Ignore statements that are mainly mentioning strengths points. D

In [17]:
client.datasets

name,id,workspace_id,updated_at
review_evaluation_qhBkNx6n,939b572d-f4a4-4404-bdbc-e4d585cbaaea,b752e2ae-6637-446e-840e-0355a1a05091,2025-03-19T11:19:48.933865
review_evaluation_cYVX6CPX,0572f958-a23d-45e8-aed3-cf61a2b6f60a,bf9821d3-6163-44a9-9faf-085e2f1a112f,2025-03-19T11:20:08.632003
review_evaluation_whnmAGJS,4d3a9990-55c0-4612-92ea-ecfe42d345e0,c3f6ad99-14e6-4d51-95ca-eee206f8206d,2025-03-19T11:20:28.405854
review_evaluation_d1clt3oE,f54ddde3-3e30-4e6d-afe6-d70a714af7dc,a1cea484-5df1-45dd-b518-878f0ee0db49,2025-03-19T11:20:47.560159
review_evaluation_RRgk02HS,c1ad7622-a2aa-4715-b911-9785b02be411,2d6e54b0-af5c-44fa-aa78-5b8c73782120,2025-03-19T11:21:07.192435
review_evaluation_tim,383f13fc-62b6-4320-840e-5270e8a024dc,3a5a73d8-c718-4d42-b515-072c6d551c38,2025-03-19T11:21:24.218432
review_evaluation_boda,ab76ff0d-cc81-45a7-8034-426fe507e68a,72e26036-d320-4593-bf6d-132904cd4bfb,2025-03-19T11:21:41.337130
